In [1]:
import pandas as pd

In [2]:
BinaryConf = pd.read_csv("BinaryConf.csv")
BinaryIV = pd.read_csv("BinaryIV.csv")
ContConf = pd.read_csv("ContConf.csv")
ContIV = pd.read_csv("ContIV.csv")

In [3]:
BinaryConf

,ATE_OLS-0.99,ATE_OLS-0.98,ATE_OLS-0.95,ATE_causaloptim,PNS_causaloptim,ATE_autobound,PNS_autobound,ATE_entropybounds-0.80,ATE_entropybounds-0.20,ATE_entropybounds-0.10,...,PNS_entropybounds-0.10,PNS_entropybounds-trueTheta,ATE_entropybounds-trueTheta,PNS_entropybounds-randomTheta,ATE_entropybounds-randomTheta,ATE_zaffalonbounds,PNS_zaffalonbounds,ATE_tianpearl,PNS_tianpearl,ATE_manski
0,5.71793,5.237387,5.294423,237.32188,268.546403,73.906253,74.2318,70.168989,86.538314,90.580307,...,50.537993,39.363327,72.977925,44.221858,77.926381,1918.245485,1917.200429,0.015701,0.013864,0.015285


In [ ]:
import numpy as np

# Get the set of all algorithm names (column names)
algorithms = set(BinaryConf.columns) | set(BinaryIV.columns) | set(ContConf.columns) | set(ContIV.columns)

# Prepare the table
rows = []
for algo in sorted(algorithms):
    vals = []
    for df in [BinaryConf, BinaryIV, ContConf, ContIV]:
        if algo in df.columns:
            val = df[algo].iloc[0]
            vals.append(val)
        else:
            vals.append(np.nan)
    avg = np.nanmean(vals)
    rows.append([algo] + vals + [avg])

# Create DataFrame
summary_df = pd.DataFrame(rows, columns=["Algorithm", "BinaryConf", "BinaryIV", "ContConf", "ContIV", "Average"])
summary_df

,Algorithm,BinaryConf,BinaryIV,ContConf,ContIV,Average
0,ATE_2SLS-0.95,NaN,19.0,NaN,20.0,20
1,ATE_2SLS-0.98,NaN,20.0,NaN,20.0,20
2,ATE_2SLS-0.99,NaN,19.0,NaN,20.0,20
3,ATE_OLS-0.95,5.0,NaN,5.0,NaN,5
4,ATE_OLS-0.98,5.0,NaN,5.0,NaN,5
5,ATE_OLS-0.99,6.0,NaN,5.0,NaN,6
6,ATE_autobound,74.0,96.0,NaN,NaN,85
7,ATE_autobound--binned,NaN,NaN,76.0,87.0,82
8,ATE_causaloptim,237.0,531.0,NaN,NaN,384
9,ATE_causaloptim--binned,NaN,NaN,326.0,575.0,450


In [9]:
def summary_df_to_latex(summary_df):
    import math
    def escape_underscore(name):
        return name.replace('_', r'\_')
    
    lines = []
    header = "Algorithm & BinaryConf & BinaryIV & ContConf & ContIV & Average \\\\"
    lines.append("\\begin{tabular}{lccccc}")
    lines.append("\\toprule")
    lines.append(header)
    lines.append("\\midrule")
    for _, row in summary_df.iterrows():
        algo = f"\\texttt{{{escape_underscore(row['Algorithm'])}}}"
        vals = [math.ceil(row[col]) if not pd.isna(row[col]) else "" for col in ["BinaryConf", "BinaryIV", "ContConf", "ContIV", "Average"]]
        line = f"{algo} & " + " & ".join(map(str, vals)) + r" \\ \hline"
        lines.append(line)
    lines.append("\\bottomrule")
    lines.append("\\end{tabular}")
    return "\n".join(lines)

# Example usage:
print(summary_df_to_latex(summary_df))

\begin{tabular}{lccccc}
\toprule
Algorithm & BinaryConf & BinaryIV & ContConf & ContIV & Average \\
\midrule
\texttt{ATE\_2SLS-0.95} &  & 19 &  & 20 & 20 \\ \hline
\texttt{ATE\_2SLS-0.98} &  & 20 &  & 20 & 20 \\ \hline
\texttt{ATE\_2SLS-0.99} &  & 19 &  & 20 & 20 \\ \hline
\texttt{ATE\_OLS-0.95} & 5 &  & 5 &  & 5 \\ \hline
\texttt{ATE\_OLS-0.98} & 5 &  & 5 &  & 5 \\ \hline
\texttt{ATE\_OLS-0.99} & 6 &  & 5 &  & 6 \\ \hline
\texttt{ATE\_autobound} & 74 & 96 &  &  & 85 \\ \hline
\texttt{ATE\_autobound--binned} &  &  & 76 & 87 & 82 \\ \hline
\texttt{ATE\_causaloptim} & 237 & 531 &  &  & 384 \\ \hline
\texttt{ATE\_causaloptim--binned} &  &  & 326 & 575 & 450 \\ \hline
\texttt{ATE\_entropybounds-0.10} & 91 & 87 &  &  & 89 \\ \hline
\texttt{ATE\_entropybounds-0.10--binned} &  &  & 171 & 173 & 172 \\ \hline
\texttt{ATE\_entropybounds-0.20} & 87 & 83 &  &  & 85 \\ \hline
\texttt{ATE\_entropybounds-0.20--binned} &  &  & 151 & 153 & 152 \\ \hline
\texttt{ATE\_entropybounds-0.80} & 70 & 66 &  &  